In [1]:
import timm
from fastai.vision.all import *
import cv2

import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

from utils import IMG_SIZE_X, IMG_SIZE_Y, preprocess_img, get_learner
from window_capture import WindowCapture

In [2]:
learn = get_learner('models/tiny384_2heads_160k_03off')


wincap = WindowCapture('Need for Speed™ Most Wanted')
screen = wincap.get_screenshot()

img = preprocess_img(screen)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = np.moveaxis(img, -1, 0)
learn.predict((img, torch.tensor(32.3)))[0]

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


g:\anaconda3\envs\ml\lib\site-packages\fastai\learner.py:58: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


(-17.513179779052734,
 0.18108314275741577,
 25.859399795532227,
 -4.372618675231934,
 -4.293115615844727,
 -17.583904266357422,
 34.15899658203125,
 -5.687933921813965,
 -11.480090141296387,
 -0.9243593215942383)

In [41]:
class ConvnextWithSpeed(nn.Module):
    def __init__(self, model): 
        super().__init__()
        head_layers = list(model[-1].children())
        head_layers[4] = torch.nn.Linear(in_features=1537, out_features=512, bias=False)
        self.img_body = nn.Sequential(model[:-1], nn.Sequential(*head_layers[:4]))
        self.head = nn.Sequential(*head_layers[4:])

    def forward(self, *x):
        img, speed = x
        x_img = self.img_body(img)
        x = torch.cat([x_img, speed[:, None]], dim=1)
        return self.head(x)

In [42]:
def dls_from_np(images, speeds, labels, num_train):
    num_images = images.shape[0]
    
    def pass_index(idx):
        return idx

    def get_x(i):
        return images[i], speeds[i]

    def get_y(i):
        # FOR MINIMAP ROTATION REGRESSION
        return labels[i]
        
        # FOR MINIMAP ROTATION CLASSIFICATION
        # return 0 if np.abs(labels[i]) <= 1 else np.sign(labels[i])
        
        # FOR KB INPUT CLASSIFICATION 
        # return CONVERT_INPUT.index(labels[i][2:])

    dblock = DataBlock(
        blocks=(ImageBlock, RegressionBlock),
        # blocks=(ImageBlock, CategoryBlock),
        get_items=pass_index,
        get_x=get_x,
        get_y=get_y,
        # item_tfms=[Resize((224, 224), method='squish')],
        splitter=IndexSplitter(list(range(num_train, num_images)))
        # splitter=EndSplitter(valid_pct=0.1)
        )
    # pass in a list of index
    dls = dblock.dataloaders(list(range(num_images)), shuffle=True, bs=64)

    return dls

In [103]:
def speed_loss(pred, speed, angle, ws, ad): return F.mse_loss(pred[:, 0], speed)
def angle_loss(pred, speed, angle, ws, ad): return F.mse_loss(pred[:, 1], angle)
def kb_input_loss(pred, speed, angle, ws, ad):
    return F.cross_entropy(pred[:, 2:6], ws, ignore_index=-2) + F.cross_entropy(pred[:, 6:], ad, ignore_index=-2)
    
def combine_loss(pred, speed, angle, ws, ad, reduction='none'): return speed_loss(pred, speed, angle, ws, ad) \
                                                                       + angle_loss(pred, speed, angle, ws, ad) \
                                                                       + kb_input_loss(pred, speed, angle, ws, ad)

dummy_x = np.empty(shape=(1, IMG_SIZE_Y, IMG_SIZE_X, 3), dtype=np.uint8)
dummy_speed = np.empty(shape=(1, ), dtype=np.uint8)
dummy_y = np.empty(shape=(1, ))
dls = dls_from_np(dummy_x, dummy_speed, dummy_y, num_train=1)

class ConvnextWithSpeed(nn.Module):
    def __init__(self, model): 
        super().__init__()
        head_layers = list(model[-1].children())
        head_layers[4] = torch.nn.Linear(in_features=1537, out_features=512, bias=False)
        self.img_body = nn.Sequential(model[:-1], nn.Sequential(*head_layers[:4]))
        self.head = nn.Sequential(*head_layers[4:])

    def forward(self, x):
        img, speed = x
        x_img = self.img_body(img/255)
        x = torch.cat([x_img, speed[:, None]], dim=1)
        return self.head(x)
    
# create model
model = ConvnextWithSpeed(create_timm_model('convnext_tiny_384_in22ft1k', n_out=10)[0])
learn = Learner(dls, model, loss_func=combine_loss, metrics=(speed_loss, angle_loss, kb_input_loss), cbs=GradientAccumulation(64))
# freeze the model
learn.model.img_body[0].requires_grad_(False)
for module in learn.model.img_body[0].modules():
    if isinstance(module, nn.LayerNorm):
        if hasattr(module, 'weight'):
            module.weight.requires_grad_(True)
        if hasattr(module, 'bias'):
            module.bias.requires_grad_(True)
        module.eval()
        
learn.load('tiny384_70k_allinp_v1')


wincap = WindowCapture('Need for Speed™ Most Wanted')
screen = wincap.get_screenshot()

img = preprocess_img(screen)
img = np.moveaxis(img, -1, 0)
learn.predict((img, 12))[0]
# # dl = dls.test_dl(img, torch.tensor(12))
# dl = dls.test_dl([(img, 12)])
# preds=learn.get_preds(dl=dl)
# preds[0]

# learn.summary()   

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


(-0.04791807383298874,
 1.0886561870574951,
 12.747199058532715,
 -3.7238669395446777,
 -3.34065318107605,
 -2.268181562423706,
 13.475717544555664,
 -5.935239791870117,
 -2.7679271697998047,
 0.38461923599243164)

In [128]:
dummy_x = np.empty(shape=(IMG_SIZE_Y, IMG_SIZE_X, 3), dtype=np.uint8)
dummy_speed = np.empty(shape=(1, ), dtype=np.uint16)

def get_x(x):
    return x

dblock = DataBlock(
    blocks=(ImageBlock, RegressionBlock, RegressionBlock, RegressionBlock, CategoryBlock([-2, -1, 0, 1]), CategoryBlock([-2, -1, 0, 1])),
    # get_items=lambda x: x,
    # get_x=[lambda x: x[0]] * 2,
    get_x=[ItemGetter(i) for i in range(2)],
    get_y=[ItemGetter(i) for i in range(2, 6)],
    # get_x=[get_x, get_x],
    # get_y=4*[lambda x: 1],
    # get_y=[ColReader('speed_diff'), ColReader('angle_diff'), ColReader('ws'), ColReader('ad')],
    n_inp=2
    )
dls = dblock.dataloaders([[dummy_x, dummy_speed, 1, 1, 1, 1]])

class ConvnextWithSpeed(nn.Module):
    def __init__(self, model): 
        super().__init__()
        head_layers = list(model[-1].children())
        head_layers[4] = torch.nn.Linear(in_features=1537, out_features=512, bias=False)
        self.img_body = nn.Sequential(model[:-1], nn.Sequential(*head_layers[:4]))
        self.head = nn.Sequential(*head_layers[4:])

    def forward(self, *x):
        img, speed = x
        x_img = self.img_body(img/255)
        x = torch.cat([x_img, speed[:, None]], dim=1)
        return self.head(x)
    
# create model
model = ConvnextWithSpeed(create_timm_model('convnext_tiny_384_in22ft1k', n_out=10)[0])
learn = Learner(dls, model, loss_func=combine_loss, metrics=(speed_loss, angle_loss, kb_input_loss), cbs=GradientAccumulation(64))
# freeze the model
learn.model.img_body[0].requires_grad_(False)
for module in learn.model.img_body[0].modules():
    if isinstance(module, nn.LayerNorm):
        if hasattr(module, 'weight'):
            module.weight.requires_grad_(True)
        if hasattr(module, 'bias'):
            module.bias.requires_grad_(True)
        module.eval()
        
learn.load('tiny384_70k_allinp_v1')


wincap = WindowCapture('Need for Speed™ Most Wanted')
screen = wincap.get_screenshot()

img = preprocess_img(screen)
# img = np.moveaxis(img, -1, 0)
learn.predict([img, 12, 0, 0, 0, 0])[0]

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


g:\anaconda3\envs\ml\lib\site-packages\fastai\learner.py:58: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


AssertionError: Match length mismatch

In [ ]:
head_layers = list(learn.model[-1].children())
nn.Sequential(*head_layers[:3])
# # head_layers.insert(3, torch.cat([img, ], dim=1))
head_layers[4] = torch.nn.Linear(in_features=1537, out_features=512, bias=False)
head_layers

[AdaptiveConcatPool2d(
   (ap): AdaptiveAvgPool2d(output_size=1)
   (mp): AdaptiveMaxPool2d(output_size=1)
 ),
 fastai.layers.Flatten(full=False),
 BatchNorm1d(1536, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Dropout(p=0.25, inplace=False),
 Linear(in_features=1537, out_features=512, bias=False),
 ReLU(inplace=True),
 BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Dropout(p=0.5, inplace=False),
 Linear(in_features=512, out_features=2, bias=False)]

In [ ]:
# list(learn.model[-1].children())[2]
# learn.model[:-1]

In [ ]:
# removed = list(learn.model[-1].children())[:-1]
# model= torch.nn.Sequential(*removed)
# learn.model[-1] = model
# learn.model[-1]